In [1]:
using JuMP
using CPLEX

Now we define the master problem :)

In [92]:
function solve_master(A,b,c,h,T,V,R)
    relaxation = 15
    nx = size(c,1)
    m = Model(CPLEX.Optimizer)
    @variable(m, x[1:nx] >= 0)
    @variable(m, theta)
    if(size(V,1) > 0)
        @objective(m, Min, sum(c[i]*x[i] for i = 1:nx) + theta)
    else
        @objective(m, Min, sum(c[i]*x[i] for i = 1:nx))
    end
    @constraint(m, A*x .== b)
    if size(R,1) > 0
        for i in 1:size(R,1)
            @constraint(m, transpose(R[i,:])*(h-T*x)<=0)
        end
    end
    if size(V,1) > 0
        for i in 1:size(V,1)
            @constraint(m, theta >= transpose(V[i,:])*(h-T*x))
        end
    end
    optimize!(m)
    sol = zeros(1, nx)
    for i = 1:nx
        sol[i] = JuMP.value(x[i])
    end
    sol = vec(sol)
    if(size(V,1)>0)
        return (objective_value(m), sol, theta,termination_status(model))
    else
        return (objective_value(m), sol, -Inf,termination_status(model))
    end
end

solve_master (generic function with 1 method)

In [ ]:
function step2(W, h, T, x, omega)
    m = Model(CPLEX.Optimizer)
    ny = size(W,2)
    
    @variable(m, y[1:ny] >= 0)
    @constraint(m, constraint, W*y - h + T*x_t .== 0)
    @objective(m, Min, W[omega,:] * y)
    optimize!(m);
    
    sol = zeros(1, ny)
    for i = 1:ny
        sol[i] = JuMP.value(y[i])
    end
    sol = vec(sol)
    
    return (objective_value(m), sol, dual.(constraint), termination_status(model))
end

In [ ]:
function step3()
    

In [2]:
function main(A, b, c, T, W, h, q, p)
    h = []
    T = []
    V = []
    S = []
    
    k = 0
    val_k, x_k, theta_k = 0, 0, 0

    while (k<10000000000)
        
        (val_k, x_k, theta_k, termination_status) = solve_master(A, b, c, h, T, V, S)
    
        if temination_status == INFEASIBLE
            return (-1, -1, -1)
        end
        
        V_temp = V
        status = 0
        
        for w in 1:size(p,2)
            (pi_k, y_k, sigma_k, termination_status) = step2(W, h, T, x, w)
            
            if temination_status == INFEASIBLE
                S_k = [S_k; sigma_k]
                k += 1
                status = 1
                break
            else
                V_temp = [V_temp p*pi_k]
            end
        end
        if(status == 0)
            V_new = [V; V_temp]
            if(V_new == V)
                return (x_k, y_k, k)
            else
                k += 1
                V = V_new
            end
        end
    end
        
            
            
        
        
    
        
    
    

solve_master (generic function with 1 method)

In [7]:
d = [500 100; 300 300]
q = [-24 -28; -28 -32]
p = [0.4 0.6]

A = [1 1; -1 0; 0 -1]
b = [120; -40; -20]
c = [100 150]

T = [-60 0; 0 -80; 0 0; 0 0]
W = [6 10; 8 5; 1 0; 0 1]
h = [0 0; 0 0; d[1,1] d[2,1]; d[1,2] d[2,2]]

(x,y, k) = main(A, b, c, T, W, h, q, p)

println(x)
println(y)
println(k)


Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Infeasibility row 'c1':  0  = 60.
Presolve time = 0.00 sec. (0.00 ticks)


LoadError: Result index of attribute MathOptInterface.VariablePrimal(1) out of bounds. There are currently 0 solution(s) in the model.